In [1]:
import os
import datetime

from vllm.steer_vectors.request import SteerVectorRequest
from vllm import LLM, SamplingParams

from easysteer.steer import StatisticalControlVector

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

/home/meixinyu/anaconda3/envs/easysteer/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


INFO 07-12 16:55:37 [__init__.py:244] Automatically detected platform cuda.


In [4]:
model_path = "/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/"
# vector_path="/home/meixinyu/EasySteer/vectors/thinking_switch_pca_MATH-500.gguf" #change to your path
vector_path="GSM8K.gguf" #change to your path

control_vector = StatisticalControlVector.import_gguf(vector_path)
print(control_vector)

StatisticalControlVector(model_type='qwen2.5', method='pca', directions={0: memmap([-0.01155342,  0.0363053 ,  0.02025236, ...,  0.02584847,
        -0.0329859 ,  0.04861534], shape=(1536,), dtype=float32), 1: memmap([0.01450336, 0.08827824, 0.04818591, ..., 0.01842126, 0.01320503,
        0.00258686], shape=(1536,), dtype=float32), 2: memmap([ 0.04029601,  0.02029571,  0.05631472, ..., -0.03308351,
        -0.00486132,  0.00283333], shape=(1536,), dtype=float32), 3: memmap([0.02305163, 0.01091879, 0.03863912, ..., 0.00034204, 0.00315652,
        0.00164229], shape=(1536,), dtype=float32), 4: memmap([ 0.03933137,  0.00631999, -0.00654727, ...,  0.0554505 ,
         0.01200243,  0.03885178], shape=(1536,), dtype=float32), 5: memmap([ 0.05188176,  0.01892056, -0.0013076 , ...,  0.03783435,
         0.03347067,  0.01241224], shape=(1536,), dtype=float32), 6: memmap([ 0.07023279,  0.00206189, -0.00402233, ...,  0.0294473 ,
         0.01383684, -0.00680031], shape=(1536,), dtype=float32), 7

In [5]:
steer_vector_request_pos = SteerVectorRequest(
            steer_vector_name="thinking_switch_pos",
            steer_vector_id=1,
            steer_vector_local_path=vector_path,
            scale=16.0,
            prefill_trigger_tokens="-1",
            prefill_trigger_positions=[-1],
            generate_trigger_tokens="-1",
            debug=True,
            algorithm='direct'
        )
steer_vector_request_neg = SteerVectorRequest(
            steer_vector_name="thinking_switch_neg",
            steer_vector_id=2,
            steer_vector_local_path=vector_path,
            scale=-16.0,
            prefill_trigger_tokens="-1",
            prefill_trigger_positions=[-1],
            generate_trigger_tokens="-1",
            debug=True,
            algorithm='direct'
        )

In [6]:
sampling_params = SamplingParams(temperature=0.0,max_tokens=1280)
prompt_template = "<|im_start|>user\n%s<|im_end|>\n<|im_start|>assistant\n"
# prompt = "Find the constant term in the expansion of $$\\left(10x^3-\\frac{1}{2x^2}\\right)^{5}$$"
prompt = "Darrell and Allen's ages are in the ratio of 7:11. If their total age now is 162, calculate Allen's age 10 years from now."
input = prompt_template % prompt

In [7]:
llm = LLM(model=model_path, enable_steer_vector=True, tensor_parallel_size=1)
output_pos = llm.generate(
                input,
                sampling_params,
                steer_vector_request=steer_vector_request_pos
            )
generated_text_pos = output_pos[0].outputs[0].text

output_neg = llm.generate(
                input,
                sampling_params,
                steer_vector_request=steer_vector_request_neg
            )
generated_text_neg = output_neg[0].outputs[0].text

INFO 07-12 16:58:50 [config.py:841] This model supports multiple tasks: {'embed', 'generate', 'classify', 'reward'}. Defaulting to 'generate'.
INFO 07-12 16:58:50 [config.py:1472] Using max model len 32768
WARNING 07-12 16:58:50 [arg_utils.py:1770] --enable-steer-vector is not supported by the V1 Engine. Falling back to V0. 


2025-07-12 16:58:50,747	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 07-12 16:58:50 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294.d20250712) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable_any_whitespace=False, disable_additional_properties=False, reasoning_backend=''), observability_config=ObservabilityConfig(show_hidden_metrics_for_version=None, otlp_traces_endpoint=None, collect_detailed_traces=None), seed=0, served_model_name=/data/zju-46/she

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.10it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.10it/s]


INFO 07-12 16:58:53 [default_loader.py:272] Loading weights took 1.01 seconds


...


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[Position Resolution] Sample 0 (length 47): positions [-1] -> absolute positions [46]
[DirectAlgorithm] Direct positions per sample: [-1] -> [46]
[DirectAlgorithm] Layer 0: Prefill - Processing 47 tokens from 1 samples
[DirectAlgorithm] Applying transformation to 1 token positions: [46]
[Position Resolution] Sample 0 (length 47): positions [-1] -> absolute positions [46]
[DirectAlgorithm] Direct positions per sample: [-1] -> [46]
[DirectAlgorithm] Layer 1: Prefill - Processing 47 tokens from 1 samples
[DirectAlgorithm] Applying transformation to 1 token positions: [46]
[Position Resolution] Sample 0 (length 47): positions [-1] -> absolute positions [46]
[DirectAlgorithm] Direct positions per sample: [-1] -> [46]
[DirectAlgorithm] Layer 2: Prefill - Processing 47 tokens from 1 samples
[DirectAlgorithm] Applying transformation to 1 token positions: [46]
[Position Resolution] Sample 0 (length 47): positions [-1] -> absolute positions [46]
[DirectAlgorithm] Direct positions per sample: [-1

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

[Position Resolution] Sample 0 (length 47): positions [-1] -> absolute positions [46]
[DirectAlgorithm] Direct positions per sample: [-1] -> [46]
[DirectAlgorithm] Layer 0: Prefill - Processing 47 tokens from 1 samples
[DirectAlgorithm] Applying transformation to 1 token positions: [46]
[Position Resolution] Sample 0 (length 47): positions [-1] -> absolute positions [46]
[DirectAlgorithm] Direct positions per sample: [-1] -> [46]
[DirectAlgorithm] Layer 1: Prefill - Processing 47 tokens from 1 samples
[DirectAlgorithm] Applying transformation to 1 token positions: [46]
[Position Resolution] Sample 0 (length 47): positions [-1] -> absolute positions [46]
[DirectAlgorithm] Direct positions per sample: [-1] -> [46]
[DirectAlgorithm] Layer 2: Prefill - Processing 47 tokens from 1 samples
[DirectAlgorithm] Applying transformation to 1 token positions: [46]
[Position Resolution] Sample 0 (length 47): positions [-1] -> absolute positions [46]
[DirectAlgorithm] Direct positions per sample: [-1

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.13it/s, est. speed input: 53.40 toks/s, output: 80.67 toks/s]


In [8]:
print(generated_text_pos+"\n\n")
print(generated_text_neg)

Certainly! Let's solve the problem step by step.

1. **Define the variables:**
   Let \( D \) be Darrell's age and \( A \) be Allen's age.

2. **Set up the ratio:**
   According to the problem, the ratio of Darrell's age to Allen's age is 7:11. This can be written as:
   \[
   \frac{D}{A} = \frac{7}{11}
   \]

3. **Express Darrell's age in terms of Allen's age:**
   From the ratio, we can express Darrell's age as:
   \[
   D = \frac{7}{11}A
   \]

4. **Set up the equation for their total age:**
   The problem states that their total age now is 162. Therefore, we can write:
   \[
   D + A = 162
   \]

5. **Substitute Darrell's age into the total age equation:**
   Substitute \( D = \frac{7}{11}A \) into the equation \( D + A = 162 \):
   \[
   \frac{7}{11}A + A = 162
   \]

6. **Combine the terms:**
   To combine the terms, we need a common denominator. The common denominator for 11 and 1 is 11:
   \[
   \frac{7A}{11} + \frac{11A}{11} = 162
   \]
   \[
   \frac{18A}{11} = 162
   \]

7. 

In [9]:
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
filename=f'fractreason_GSM8K_{timestamp}.txt'
# filename=f'../../temp/fractreason_GSM8K_{timestamp}.txt'
with open(filename, 'w', encoding='utf-8') as f:
    f.write("=== Positive: Slow thinking ===\n")
    f.write(generated_text_pos + "\n\n")
    f.write("=== Negative: Fast thinking ===\n")
    f.write(generated_text_neg + "\n")